In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Solution_securite_perso/fusion

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Solution_securite_perso/fusion


In [ ]:
%cd /content/drive/MyDrive/Solution_securite_perso/fusion

/content/drive/MyDrive/Solution_securite_perso/fusion


In [ ]:
%pip install pandas scikit-learn xgboost joblib pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 22.0 MB/s eta 0:00:00


In [ ]:
# Cell 1: Imports and Mock Setup
import pandas as pd
from datetime import datetime
from unittest.mock import MagicMock

from process_capture import process_capture
from fusion_core import fuse_alerts, display_outputs, trigger_alert, DEFAULT_WEIGHTS

# Mock database collections
ts_collection = MagicMock()
geo_collection = MagicMock()
users_collection = MagicMock()
ts_collection.find.return_value = [
    {"user_id": "user1", "latitude": 36.8065, "longitude": 10.1815, "timestamp": datetime.now(), "hour": 12}]
"""# Mock model functions
def capture_and_store(user_id, lat, lon, ts_collection, geo_collection, users_collection):
    return datetime.now()

def load_threshold_model(user_id):
    return None

def predict_threshold(model, features):
    return 0.05

def detect_user_anomalies(lat, lon, hour, weekday, month, user_id, ts_collection, prob_threshold):
    return 0.0, 0.6  # loc_anomaly, time_anomaly

def load_incident_model(user_id):
    return None, None

def predict_incident(model, scaler, loc_anomaly, time_anomaly):
    return 0.5

def predict_crime_risk(lat, lon):
    return 0.7

def detect_movement_anomaly(user_id, ts_collection, lat, lon, now):
    return 0.8

def predict_audio_stress(user_id):
    return 0.4

def detect_emergency_keyword(user_id):
    return 0.0

def insert_user_alert(users_collection, user_id, loc_anomaly, time_anomaly, incident_prob, trigger_alert):
    print(f"Stored alert for {user_id}")"""
#INSTEAD HERE WE NEED TO CALL OUR MODELS.


'# Mock model functions\ndef capture_and_store(user_id, lat, lon, ts_collection, geo_collection, users_collection):\n    return datetime.now()\n\ndef load_threshold_model(user_id):\n    return None\n\ndef predict_threshold(model, features):\n    return 0.05\n\ndef detect_user_anomalies(lat, lon, hour, weekday, month, user_id, ts_collection, prob_threshold):\n    return 0.0, 0.6  # loc_anomaly, time_anomaly\n\ndef load_incident_model(user_id):\n    return None, None\n\ndef predict_incident(model, scaler, loc_anomaly, time_anomaly):\n    return 0.5\n\ndef predict_crime_risk(lat, lon):\n    return 0.7\n\ndef detect_movement_anomaly(user_id, ts_collection, lat, lon, now):\n    return 0.8\n\ndef predict_audio_stress(user_id):\n    return 0.4\n\ndef detect_emergency_keyword(user_id):\n    return 0.0\n\ndef insert_user_alert(users_collection, user_id, loc_anomaly, time_anomaly, incident_prob, trigger_alert):\n    print(f"Stored alert for {user_id}")'

In [ ]:
!ls

alerts.log	     call_models.py	  __pycache__
api.py		     custom_interface.py  test_safety_system.ipynb
audio_processing.py  fusion_core.py	  vocal_analysis.py
audio.py	     __init__.py	  wav2vec2_checkpoints
audio_test.wav	     process_capture.py


In [ ]:
import joblib
from pathlib import Path

IMPORTING THE FIRSTMODEL: HIGH RISK LOCATOR

In [ ]:



#DBSCAN artifacts
dbscan       = joblib.load( "../models/riskyzones/risky_location_model_dbscan.pkl")
cw_dbscan    = joblib.load("../models/riskyzones/cluster_weights_dbscan.pkl")
cf_dbscan    = joblib.load("../models/riskyzones/cluster_fatalities_dbscan.pkl")
cet_dbscan   = joblib.load("../models/riskyzones/cluster_event_types_dbscan.pkl")

#OPTICS artifacts
optics       = joblib.load("../models/riskyzones/risky_location_model_optics.pkl")
cw_optics    = joblib.load("../models/riskyzones/cluster_weights_optics.pkl")
cf_optics    = joblib.load("../models/riskyzones/cluster_fatalities_optics.pkl")
cet_optics   = joblib.load("../models/riskyzones/cluster_event_types_optics.pkl")
#the coordinate scaler and scaled data
scaler = joblib.load("../models/riskyzones/scaler__risky_loc.pkl")
X_scaled = joblib.load("../models/riskyzones/X_scaled.pkl")


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DBSCAN from version 1.5.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator OPTICS from version 1.5.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.5.2 when using version 1.6.1. This might lead to breaking code or invalid results.

LOADING BEHAVIORAL MODELS AND FUNCTIONS

In [ ]:
# Load models for user 1(as a prototype user)
# 2) Dynamic‐threshold model (RandomForestRegressor)
thr_model = joblib.load("../models/user1/user1_threshold_model.pkl")

# 3) Incident‐prediction model + scaler
inc_model   = joblib.load("../models/user1/user1_xgboost_incident_pred.pkl")
inc_scaler  = joblib.load("../models/user1/user1_xgboost_incident_pred_scaler.pkl")
behavioral_model = joblib.load("../models/user1/user1_optics_behavioral_clust.pkl")
scaler_behavioral = joblib.load("../models/user1/user1_scaler_behavioral_clust.pkl")


from behavioral_alerts.core.profiling import detect_user_anomalies, load_profile, build_user_profile
from behavioral_alerts.core.threshold_adjustment import load_threshold_model, predict_threshold
from behavioral_alerts.core.incident_prediction import load_incident_model, predict_incident


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.7.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RandomForestRegressor from version 1.7.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [08:47:39] WARNING: /workspace/src/collective/../data/../common/error_msg.h:80: If you are loading a serialized model (like pi

LOAADING VOCAL MODELS

In [ ]:
!pip install openai-whisper
!pip install transformers librosa numpy torch
!pip install SpeechBrain pyannote.audio soundfile

In [ ]:

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Solution_securite_perso/fusion
import gc
gc.collect()

import vocal_analysis

# Example usage
audio_path = "audio_test.wav"

# Individual functions
transcription = vocal_analysis.transcribe_audio(audio_path)
print("Transcription:", transcription)

features = vocal_analysis.audio_features(audio_path)
print("Audio Features:", features)


classification = vocal_analysis.classify_text(transcription)
print("Classification:", classification)


#full implementation
result = vocal_analysis.analyze_vocal_input(audio_path)
print("Full Analysis:", result)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Solution_securite_perso/fusion


SyntaxError: invalid syntax (vocal_analysis.py, line 153)

In [ ]:
#Comprehensive analysis
import vocal_analysis

audio_path = "audio_test.wav"
result = vocal_analysis.analyze_vocal(audio_path)
print("Full Analysis:", result)

SyntaxError: invalid syntax (vocal_analysis.py, line 153)

In [ ]:

# Cell 2: Test Scenario 1 - Normal Case
print("=== Testing Normal Case ===")
result = process_capture("user123", 40.7128, -74.0060, ts_collection, geo_collection, users_collection)
print("Result:", result)

# Cell 3: Test Scenario 2 - High-Risk Case
print("\n=== Testing High-Risk Case ===")
def detect_emergency_keyword(user_id):  # Override for high-risk
    return 1.0
def predict_audio_stress(user_id):  # Override for high-risk
    return 0.9
result = process_capture("user123", 40.7128, -74.0060, ts_collection, geo_collection, users_collection)
print("High-Risk Result:", result)

# Cell 4: Test Scenario 3 - Missing Data
print("\n=== Testing Missing Data ===")
def predict_crime_risk(lat, lon):  # Simulate missing data
    return 0.0
def detect_movement_anomaly(user_id, ts_collection, lat, lon, now):
    return 0.0
result = process_capture("user123", 40.7128, -74.0060, ts_collection, geo_collection, users_collection)
print("Missing Data Result:", result)

# Cell 5: Test Scenario 4 - Error Case
print("\n=== Testing Error Case ===")
def capture_and_store(user_id, lat, lon, ts_collection, geo_collection, users_collection):
    raise Exception("Database connection failed")
result = process_capture("user123", 40.7128, -74.0060, ts_collection, geo_collection, users_collection)
print("Error Case Result:", result)

In [ ]:
# Cell 6: Test Scenario 5 - Low-Risk Case
print("\n=== Testing Low-Risk Case ===")
def predict_crime_risk(lat, lon):
    return 0.1
def detect_movement_anomaly(user_id, ts_collection, lat, lon, now):
    return 0.1
def predict_audio_stress(user_id):
    return 0.1
def detect_emergency_keyword(user_id):
    return 0.0
def detect_user_anomalies(lat, lon, hour, weekday, month, user_id, ts_collection, prob_threshold):
    return 0.0, 0.1
def predict_incident(model, scaler, loc_anomaly, time_anomaly):
    return 0.1
result = process_capture("user123", 40.7128, -74.0060, ts_collection, geo_collection, users_collection)
print("Low-Risk Result:", result)

In [7]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Solution_securite_perso
!git add .
!git commit -m "vocal file and test done "
!git push -u origin main

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Solution_securite_perso
fatal: could not parse HEAD
Host key verification failed.
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.
